# Preprocessing

### Environment Setup

In [1]:
versioninfo()

Julia Version 1.10.0
Commit 3120989f39b (2023-12-25 18:01 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 10 × Apple M1 Pro
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
  Threads: 1 on 8 virtual cores
Environment:
  JULIA_NUM_THREADS = 


In [31]:
import Pkg
Pkg.activate(pwd())
Pkg.status()

  Activating project at `~/Library/CloudStorage/OneDrive-Personal/UCR/academics/winter_24/STAT206/project`


Status `~/Library/CloudStorage/OneDrive-Personal/UCR/academics/winter_24/STAT206/project/Project.toml`
  [324d7699] CategoricalArrays v0.10.8
  [31c24e10] Distributions v0.25.107
  [89b67f3b] ExcelFiles v1.0.0
  [da1fdf0e] FreqTables v0.4.6
  [38e38edf] GLM v1.9.0
  [09f84164] HypothesisTests v0.11.0
  [c709b415] Imbalance v0.1.5
  [add582a8] MLJ v0.20.3
  [a7f614a8] MLJBase v1.1.2
  [caf8df21] MLJGLMInterface v0.3.7
  [6ee0df7b] MLJLinearModels v0.10.0
  [d491faf4] MLJModels v0.16.16
  [03970b2e] MLJTuning v0.8.2
  [91a5bcdd] Plots v1.40.2
  [ce6b1742] RDatasets v0.7.7
  [8e980c4a] Shapefile v0.12.0
  [860ef19b] StableRNGs v1.0.1
  [2913bbd2] StatsBase v0.34.2
  [3eaba693] StatsModels v0.7.3
  [f3b207a7] StatsPlots v0.15.7
  [9a3f8284] Random


### Importing Libraries

In [3]:
# Loading Data
using LinearAlgebra, Statistics, StatsBase, HypothesisTests
using Distributions, Random, StableRNGs, RDatasets
using CSV, DataFrames, FreqTables, CategoricalArrays
using Plots, StatsPlots, Shapefile, MLJ, MLJModels, Imbalance

### Loading Data

In [29]:
# Load the the data csv file from the /data directory
df = DataFrame(CSV.File("data/telco.csv"))

# Peek at the first 5 rows of the dataframe
first(df, 5) |> pretty

┌────────────┬───────┬───────────────┬────────────┬─────────────┬──────────┬────────────────────────┬────────────┬────────────┬─────────┬────────────────┬─────────┬────────────┬───────────────┬───────────────┬────────────────┬──────────────────┬─────────────────┬───────────────┬───────────────────┬──────────────┬──────────────┬──────────────────┬────────────────┬───────────────────┬───────────────────────────┬─────────────────┬───────────────┬─────────────┬─────────────┬─────────────┬───────┬───────────────────────────────┐
│ CustomerID │ Count │ Country       │ State      │ City        │ Zip Code │ Lat Long               │ Latitude   │ Longitude  │ Gender  │ Senior Citizen │ Partner │ Dependents │ Tenure Months │ Phone Service │ Multiple Lines │ Internet Service │ Online Security │ Online Backup │ Device Protection │ Tech Support │ Streaming TV │ Streaming Movies │ Contract       │ Paperless Billing │ Payment Method            │ Monthly Charges │ Total Charges │ Churn Label │ Churn Va

### Renaming Columns for Consistency

In [28]:
# Renaming all variables to snake case
rename!(df, Symbol("CustomerID") => :customer_id)
rename!(df, Symbol("Count") => :count)
rename!(df, Symbol("Country") => :country)
rename!(df, Symbol("State") => :state)
rename!(df, Symbol("City") => :city)
rename!(df, Symbol("Zip Code") => :zip_code)
rename!(df, Symbol("Lat Long") => :lat_long)
rename!(df, Symbol("Latitude") => :latitude)
rename!(df, Symbol("Longitude") => :longitude)
rename!(df, Symbol("Gender") => :gender)
rename!(df, Symbol("Senior Citizen") => :senior_citizen)
rename!(df, Symbol("Partner") => :partner)
rename!(df, Symbol("Dependents") => :dependents)
rename!(df, Symbol("Tenure Months") => :tenure_months)
rename!(df, Symbol("Phone Service") => :phone_service)
rename!(df, Symbol("Multiple Lines") => :multiple_lines)
rename!(df, Symbol("Internet Service") => :internet_service)
rename!(df, Symbol("Online Security") => :online_security)
rename!(df, Symbol("Online Backup") => :online_backup)
rename!(df, Symbol("Device Protection") => :device_protection)
rename!(df, Symbol("Tech Support") => :tech_support)
rename!(df, Symbol("Streaming TV") => :streaming_tv)
rename!(df, Symbol("Streaming Movies") => :streaming_movies)
rename!(df, Symbol("Contract") => :contract)
rename!(df, Symbol("Paperless Billing") => :paperless_billing)
rename!(df, Symbol("Payment Method") => :payment_method)
rename!(df, Symbol("Monthly Charges") => :monthly_charges)
rename!(df, Symbol("Total Charges") => :total_charges)
rename!(df, Symbol("Churn Label") => :churn_label)
rename!(df, Symbol("Churn Value") => :churn_value)
rename!(df, Symbol("Churn Score") => :churn_score)
rename!(df, Symbol("CLTV") => :cltv)
rename!(df, Symbol("Churn Reason") => :churn_reason)

# Display the first 5 rows of the data
first(df, 5) |> pretty

ArgumentError: ArgumentError: Tried renaming :CustomerID to :customer_id, when :CustomerID does not exist in the data frame.

### Data Overview

In [6]:
# Dimensions of the data
size(df)

(7043, 33)

We can see there are 7043 rows and 33 columns in the dataset. However, some columns seem to be redundant and can be dropped.

In [26]:
# Summary of the data
describe(df) |> pretty

┌───────────────────┬────────────────────────────┬───────────────────────────────────┬────────────────────────────┬───────────────────────────────────┬──────────┬────────────────────────┐
│ variable          │ mean                       │ min                               │ median                     │ max                               │ nmissing │ eltype                 │
│ Symbol            │ Union{Nothing, Float64}    │ Any                               │ Union{Nothing, Float64}    │ Any                               │ Int64    │ Type                   │
│ Unknown           │ Union{Nothing, Continuous} │ Union{Continuous, Count, Textual} │ Union{Nothing, Continuous} │ Union{Continuous, Count, Textual} │ Count    │ Unknown                │
├───────────────────┼────────────────────────────┼───────────────────────────────────┼────────────────────────────┼───────────────────────────────────┼──────────┼────────────────────────┤
│ customer_id       │ nothing                    │ 0002-ORFB

### Checking for Missing Values

In [8]:
# Drop observations that have value equal to " "
df = df[.!(df.total_charges .== " "), :]

# Checking for missing values
missing_vals = [sum(ismissing(df[:, i])) for i in 1:size(df, 2)]

# There are no missing values so no need to drop any rows
println("There are ", sum(missing_vals), " missing values in the data")

There are 0 missing values in the data


### Removing Redundant Columns.
The rationale for removing these columns is pontificated in the final report.

In [25]:
# Columns to remove
columns_to_remove = [
  :customer_id,
  :count,
  :country,
  :state,
  :city,
  :zip_code,
  :lat_long,
  :churn_label,
  :churn_score,
  :online_security,
  :online_backup,
  :streaming_tv,
  :streaming_movies,
  :churn_reason
]

# Drop the columns_to_remove
clean_df = select(df, Not(columns_to_remove))

# Summary of the data
first(clean_df, 5) |> pretty

┌────────────┬────────────┬─────────┬────────────────┬─────────┬────────────┬───────────────┬───────────────┬────────────────┬──────────────────┬───────────────────┬──────────────┬────────────────┬───────────────────┬───────────────────────────┬─────────────────┬───────────────┬─────────────┬───────┐
│ latitude   │ longitude  │ gender  │ senior_citizen │ partner │ dependents │ tenure_months │ phone_service │ multiple_lines │ internet_service │ device_protection │ tech_support │ contract       │ paperless_billing │ payment_method            │ monthly_charges │ total_charges │ churn_value │ cltv  │
│ Float64    │ Float64    │ String7 │ String3        │ String3 │ String3    │ Int64         │ String3       │ String31       │ String15         │ String31          │ String31     │ String15       │ String3           │ String31                  │ Float64         │ String7       │ Int64       │ Int64 │
│ Continuous │ Continuous │ Textual │ Textual        │ Textual │ Textual    │ Count         │ 

In [10]:
# Data shape after cleaning
size(clean_df)

(7032, 19)

### Save the Cleaned Data

In [11]:
# Store the cleaned data into a new file
CSV.write("data/telco_cleaned.csv", clean_df)

"data/telco_cleaned.csv"

### Factorizing Categorical Columns

In [24]:
transform!(clean_df, :gender => categorical => :gender)
transform!(clean_df, :senior_citizen => categorical => :senior_citizen)
transform!(clean_df, :partner => categorical => :partner)
transform!(clean_df, :dependents => categorical => :dependents)
transform!(clean_df, :phone_service => categorical => :phone_service)
transform!(clean_df, :multiple_lines => categorical => :multiple_lines)
transform!(clean_df, :internet_service => categorical => :internet_service)
transform!(clean_df, :device_protection => categorical => :device_protection)
transform!(clean_df, :tech_support => categorical => :tech_support)
transform!(clean_df, :contract => categorical => :contract)
transform!(clean_df, :paperless_billing => categorical => :paperless_billing)
transform!(clean_df, :payment_method => categorical => :payment_method)

describe(clean_df) |> pretty

┌───────────────────┬────────────────────────────┬───────────────────────────────────────────────────────────────────────┬────────────────────────────┬───────────────────────────────────────────────────────────────────────┬──────────┬────────────────────────────────────┐
│ variable          │ mean                       │ min                                                                   │ median                     │ max                                                                   │ nmissing │ eltype                             │
│ Symbol            │ Union{Nothing, Float64}    │ Any                                                                   │ Union{Nothing, Float64}    │ Any                                                                   │ Int64    │ DataType                           │
│ Unknown           │ Union{Nothing, Continuous} │ Union{Continuous, Count, Multiclass{3}, Multiclass{2}, Multiclass{4}} │ Union{Nothing, Continuous} │ Union{Continuous, Count, Multicl

### Standardizing the Numerical Features

In [13]:
numerical_columns = [
  :latitude,
  :longitude,
  :tenure_months,
  :monthly_charges,
  :total_charges,
  :cltv
]

# Parse :total_charges to float64
clean_df[!, :total_charges] = parse.(Float64, clean_df[!, :total_charges])

# Loop through the numerical columns and convert them to float64
for col in numerical_columns
  clean_df[!, col] = convert.(Float64, clean_df[!, col])
end

# Standardize the numerical columns
for col in numerical_columns
  μ = mean(clean_df[!, col])
  σ = std(clean_df[!, col])
  clean_df[!, col] = (clean_df[!, col] .- μ) ./ σ
end

### One-Hot Encoding the Categorical Features

In [14]:
hot_encode = fit!(machine(OneHotEncoder(), clean_df), verbosity = 0)

# apply the dummy coding scheme; note that we qualify `transform`
clean_df_hot_encoded = MLJ.transform(hot_encode, clean_df)

# check
schema(clean_df_hot_encoded)

┌────────────────────────────────────────┬────────────┬─────────┐
│ names                                  │ scitypes   │ types   │
├────────────────────────────────────────┼────────────┼─────────┤
│ latitude                               │ Continuous │ Float64 │
│ longitude                              │ Continuous │ Float64 │
│ gender__Female                         │ Continuous │ Float64 │
│ gender__Male                           │ Continuous │ Float64 │
│ senior_citizen__No                     │ Continuous │ Float64 │
│ senior_citizen__Yes                    │ Continuous │ Float64 │
│ partner__No                            │ Continuous │ Float64 │
│ partner__Yes                           │ Continuous │ Float64 │
│ dependents__No                         │ Continuous │ Float64 │
│ dependents__Yes                        │ Continuous │ Float64 │
│ tenure_months                          │ Continuous │ Float64 │
│ phone_service__No                      │ Continuous │ Float64 │
│ phone_se

### Checking for Outliers or Anomalies

In [15]:
describe(clean_df_hot_encoded) |> pretty

┌───────────────────────────────────────────┬──────────────┬──────────────────────────┬────────────┬──────────────────────────┬──────────┬──────────┐
│ variable                                  │ mean         │ min                      │ median     │ max                      │ nmissing │ eltype   │
│ Symbol                                    │ Float64      │ Real                     │ Float64    │ Real                     │ Int64    │ DataType │
│ Unknown                                   │ Continuous   │ Union{Continuous, Count} │ Continuous │ Union{Continuous, Count} │ Count    │ Unknown  │
├───────────────────────────────────────────┼──────────────┼──────────────────────────┼────────────┼──────────────────────────┼──────────┼──────────┤
│ latitude                                  │ 7.76019e-16  │ -1.51763                 │ 0.0441632  │ 2.31211                  │ 0        │ Float64  │
│ longitude                                 │ -7.14585e-15 │ -2.08666                 │ 0.0295724  │

### Save the Preprocessed Data

In [16]:
# Store the one-hot encoded data into a new file
CSV.write("data/telco_standardized.csv", clean_df_hot_encoded)

"data/telco_standardized.csv"

### Address Class Imbalance
We take into account any potential class imbalance when splitting the data. Two common approaches to address class imbalance that we will use are:
1. Undersampling the Majority Class
2. Oversampling the Minority Class using Synthetic Minority Over-sampling Technique (SMOTE)

We will use both approaches and create two different datasets to compare the results.

In [17]:
# Check if there is class imbalance for churn_value
churn_value_full = mean(clean_df.churn_value)

0.26578498293515357

There is clearly a class imbalance in the target variable. We will use sampling to address this issue.

#### 1. Undersampling the Majority Class

In [18]:
# Select observations so that the classes are balanced
df_churn = clean_df_hot_encoded[clean_df_hot_encoded.churn_value .== 1, :]
df_no_churn = clean_df_hot_encoded[clean_df_hot_encoded.churn_value .== 0, :]

# Number of observations in each class
n_churn = size(df_churn, 1)
n_no_churn = size(df_no_churn, 1)

# Randomly select n_churn observations from df_no_churn
df_no_churn = df_no_churn[shuffle(1:n_no_churn)[1:n_churn], :]

# Combine the two datasets
df_balanced = vcat(df_churn, df_no_churn)

# Shuffle the rows
df_balanced = df_balanced[shuffle(1:2*n_churn), :]

# Summary of the balanced data
size(df_balanced)

(3738, 38)

In [19]:
# Sanity Check for class imbalance
count_by_churn_value = countmap(df_balanced.churn_value)

Dict{Int64, Int64} with 2 entries:
  0 => 1869
  1 => 1869

### Store the balanced data in a new CSV file

In [20]:
# Save the balanced data into a new file
CSV.write("data/telco_balanced_undersampling.csv", df_balanced)

"data/telco_balanced_undersampling.csv"

#### 2. Oversampling the Minority Class using SMOTE

In [21]:
# Prepare the data

# Create a matrix convert the dataframe to a matrix Float64
X = Matrix(clean_df_hot_encoded[!, Not(:churn_value)])
y = clean_df_hot_encoded[!, :churn_value]

# Apply SMOTE, we will use the default k = 5
X_smote, y_smote = smote(X, y, k = 5)

df_balanced_smote = DataFrame(X_smote, names(clean_df_hot_encoded[!, Not(:churn_value)]))
df_balanced_smote[!, :churn_value] = y_smote

# Summary of the SMOTE data
size(df_balanced_smote)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


(10326, 38)

In [22]:
# Sanity Check for class imbalance
count_by_churn_value_smote = countmap(df_balanced_smote.churn_value)

Dict{Int64, Int64} with 2 entries:
  0 => 5163
  1 => 5163

# Save the balanced SMOTE data in a new CSV file

In [23]:
# Save the balanced data into a new file
CSV.write("data/telco_balanced_oversampling.csv", df_balanced_smote)

"data/telco_balanced_oversampling.csv"